In [2]:
import numpy as np

In [8]:
array1 = np.array([[1, 2, 3], [4, 5, 6]])
print(array1)
print("")
array2 = np.array([[7, 8, 9], [10, 11, 12]])
print(array2)
print("")

print(np.concatenate((array1, array2), axis=0))
print("")
print(np.concatenate((array1, array2), axis=1))

[[1 2 3]
 [4 5 6]]

[[ 7  8  9]
 [10 11 12]]

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]]
